In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from ydata_profiling import ProfileReport
import src.data.make_dataset as data
import glob
import json

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [3]:
df_raw = pd.concat(
    (
        pd.read_excel(archivo, usecols=data.COLS_A_OCUPAR.keys())
        for archivo in glob.glob("../data/raw/diagnosticos/*.xlsx")
    )
)
df_raw = df_raw.drop(columns=data.COLS_A_ELIMINAR + ["Rut Paciente", "Rut Profesional"])


In [4]:
df_raw["Código Diagnóstico"] = df_raw["Código Diagnóstico"].astype(str)


In [5]:
df_limpia = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";"
)

# Analisis exploratorio previo al preprocesamiento

In [6]:
report = ProfileReport(df_raw)
report.to_file("../reports/1.0-jrb-exploratory-data-analysis.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Analisis de Columna Codigo Diagnostico

In [7]:
largo_codigos = df_raw["Código Diagnóstico"].str.len().value_counts(normalize=True, dropna=False)
display(largo_codigos * 100)


5    58.598147
3    27.038000
7     9.194450
2     1.983357
6     1.550353
4     1.053433
1     0.581900
8     0.000360
Name: Código Diagnóstico, dtype: float64

- El 58.6% de los codigos presenta largo 5. Los codigos con largo 5 son del tipo: "C38.4" o "C34_1", por
lo que es el formato deseable

In [8]:
df_raw[df_raw["Código Diagnóstico"].str.len() == 5]["Código Diagnóstico"].sample(10)


154176    I47.2
234643    G47.3
166330    J44.9
3982      400-9
45180     R06.0
67833     J84.9
102970    E84.0
79728     J44.9
27826     Q67.6
200962    J43.2
Name: Código Diagnóstico, dtype: object


- El 27.0% de los codigos presenta largo 3. Los codigos con largo 3 son del tipo: "R05", "C34", por
lo que corresponde a la Categoria del diagnostico.


In [9]:
largo_3 = df_raw[df_raw["Código Diagnóstico"].str.len() == 3][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_3)

Código Diagnóstico  Nombre Diagnóstico                                               
I99                 Otros trastornos y los no especificados del sistema circulatorio     29603
J47                 Bronquiectasia                                                       15305
I50                 Insuficiencia cardíaca                                               13381
C34                 Tumor maligno de los bronquios y del pulmón                          11886
J84                 Otras enfermedades pulmonares intersticiales                         11185
                                                                                         ...  
I13                 Hipertensión cardíaca y enfermedad renal.                                1
R96                 Otras muertes súbitas de causa desconocida                               1
H60                 Otitis externa                                                           1
H55                 Nistagmo y otros movimientos oculares i

- El 9.19% de los codigos presenta largo 7. Los codigos con largo 7 son del tipo: "J44.8.4" o "E10-E16".

In [10]:
largo_7 = df_raw[df_raw["Código Diagnóstico"].str.len() == 7]["Código Diagnóstico"].sample(10)
display(largo_7)


182320    J45.9.1
128624    J45.0.1
288462    J43.1.2
89344     I20-I25
183335    J45.1.1
70745     J45.0.1
128046    J45.0.1
160026    J44.8.4
72960     J45.0.3
102392    J45.0.2
Name: Código Diagnóstico, dtype: object

- El 1.98% de los codigos presenta largo 2. Los codigos con largo 2 son del tipo: "55", por lo que
tienen una notacion de diagnostico deficiente. Sin embargo, estos codigos se relacionan a un
unico "Nombre Diagnostico". Esto puede permitir recodificar estos codigos de forma mas facil.

In [11]:
largo_2 = df_raw[df_raw["Código Diagnóstico"].str.len() == 2][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_2)

Código Diagnóstico  Nombre Diagnóstico                    
55                  Fibrosis pulmonar                         4335
79                  Trasplante  Pulmon                        2788
78                  Trasplante Cardiaco                       1197
41                  Nódulo pulmonar                            842
20                  Bronquiectasias                            363
54                  Enfermedad pulmonar difusa                 342
42                  Tumor pulmonar                             113
77                  Asma Bronquial en el Adulto (Sospecha)     110
74                  Tabaquismo                                 100
64                  Hipertensión pulmonar primaria              99
17                  Neumonitis por hipersensibilidad            98
49                  Disnea en estudio                           79
44                  Cáncer pulmonar                             53
21                  Bronquiectasias infectadas                  50
59 

- El 1.55% de los codigos presenta largo 6. Los codigos con largo 6 son del tipo: "300-23". Tambien tienen
una relacion 1:1 entre Código Diagnóstico y Nombre Diagnóstico.

In [12]:
largo_6 = df_raw[df_raw["Código Diagnóstico"].str.len() == 6][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_6)

Código Diagnóstico  Nombre Diagnóstico                                                             
300-23              Tetralogía de Fallot (TOF)                                                         2523
300-32              Ventriculo Unico Funcional                                                         1385
300-28              Transposición Grandes Arterias (discordancia de la conexión ventriculoarterial)     848
300-29              Transposición Grandes Arterias + CIV                                                519
300-12              Doble Salida VD (DSVD)                                                              517
                                                                                                       ... 
902-12              Preexcitación ventricular                                                             2
I51.14              CIV Sub Aórtica (tipo fallot)                                                         2
902-27              Macroreentrada p

- El 1.05% de los codigos presenta largo 4 y son del tipo: "81-4" o "I32*"

In [13]:
largo_4 = df_raw[df_raw["Código Diagnóstico"].str.len() == 4][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_4)

Código Diagnóstico  Nombre Diagnóstico                                     
81-4                Otro                                                       4553
88-1                Otro:                                                       300
81-6                Estenosis Pulmonar Valvular                                 254
91-3                Enfermedad Coronaria 3V                                     251
85-6                Cardiomiopatía Isquemica                                    126
91-1                Enfermedad Coronaria 1V                                      80
85-3                Enfermedad Coronaria (cardiopatía coronaria)                 73
91-2                Enfermedad Coronaria 2V                                      66
91-5                Aneurisma Aorta Ascendnte (sin mención de ruptura)           43
91-6                Aneurisma Raíz Aórtica (sin mención de ruptura)              25
81-5                Estenosis Pulmonar Subvalvular (Infundibular)                16


- El 0.58% de los codigos presenta largo 1, y son del tipo "5" o "6"

In [14]:
largo_1 = df_raw[df_raw["Código Diagnóstico"].str.len() == 1][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_1)

Código Diagnóstico  Nombre Diagnóstico          
5                   Otro                            3072
6                   Ex TBC pulmonar                  107
2                   Crisis asmática                   23
7                   TBC pulmonar                      23
8                   TBC pulmonar multiresistente       5
9                   Micobacteriosis atipica            2
dtype: int64

- El 0.00036% de los codigos presenta largo 8, y son del tipo "Z00- Z13"

In [15]:
largo_8 = df_raw[df_raw["Código Diagnóstico"].str.len() == 8][
    ["Código Diagnóstico", "Nombre Diagnóstico"]
].value_counts()
display(largo_8)

Código Diagnóstico  Nombre Diagnóstico                                  
Z00- Z13            PRUEBAS PARA ACLARAR O INVESTIGAR PROBLEMAS DE SALUD    2
dtype: int64

## Preprocesamiento de Diagnostico

- Teniendo en cuenta los resultados previos, una propuesta tentativa para el preprocesamiento
de los codigos de diagnosticos es:

    1. Eliminar el simbolo "."
    2. Eliminar los espacios
    2. A los que tienen largo 3, agregar una X al final hasta tener un largo 4
    3. A los que tienen largo 7, se separaran en diags con largo 5 (ej: J4591) y largo 7 (ej: E65-E68).
    Los que sigan teniendo largo 7 corresponderan a los Capitulos, por lo que sera necesario observar
    que hacer con esos diagnosticos (ya que son muy poco especificos)
    4. A los que tienen largo 2, 6, 4 y 1 se puede crear un diccionario para transformar el codigo a un
    codigo cie

# Analisis de base preprocesada

## Analisis de Columna Codigo Diagnostico

In [16]:
largos = df_limpia["codigo_diagnostico"].str.len().value_counts().index
descripcion_largos = df_limpia["codigo_diagnostico"].str.len().value_counts(normalize=True, dropna=False)
display(descripcion_largos * 100)

4    95.285205
2     1.829785
7     1.451923
1     0.620100
6     0.561707
5     0.251280
Name: codigo_diagnostico, dtype: float64

Luego del preprocesamiento, los porcentajes de tipos de codigos fueron los siguientes:

- El 58.6% de los codigos de la base tienen largo 4
- El 27.4% de los codigos de la base tienen largo 3
- El 9.0% de los codigos de la base tienen largo 5

Esto ya abarca el 95% de toda la base de datos

In [17]:
resultados = {}
for largo in largos:
    filtro_por_largo = df_limpia["codigo_diagnostico"].str.len() == largo

    resumen_diags_con_largo = df_limpia[filtro_por_largo][
        ["codigo_diagnostico", "nombre_diagnostico"]
    ].value_counts()

    resultados[f"largo_{largo}"] = resumen_diags_con_largo

    print(f"Los codigos con largo {largo} tienen la siguiente forma:")
    print(resumen_diags_con_largo.sample(30, replace=True).index.get_level_values(0))
    print()

Los codigos con largo 4 tienen la siguiente forma:
Index(['Z953', 'I251', 'B022', 'G711', 'C445', 'D190', 'J180', 'Z951', 'C433',
       'I520', 'R049', 'D849', 'Q289', 'Z913', 'J91X', 'J189', 'H438', 'I132',
       'Q246', 'J985', 'E168', 'Q76X', 'A28X', 'H44X', 'J342', 'M898', 'D259',
       'E063', 'M848', 'B580'],
      dtype='object', name='codigo_diagnostico')

Los codigos con largo 2 tienen la siguiente forma:
Index(['30', '35', '62', '11', '61', '78', '55', '24', '10', '61', '58', '68',
       '11', '34', '17', '45', '17', '80', '18', '30', '41', '13', '17', '62',
       '17', '37', '23', '28', '79', '55'],
      dtype='object', name='codigo_diagnostico')

Los codigos con largo 7 tienen la siguiente forma:
Index(['D50-D53', 'C40-C41', 'E00-E07', 'C43-C49', 'T15-T19', 'C50-C58',
       'Q30-Q34', 'Q20-Q28', 'D80-D89', 'J80-J84', 'K40-K46', 'H15-H22',
       'Q20-Q28', 'M00-M25', 'Q20-Q28', 'E00-E07', 'M80-M94', 'N10-N16',
       'G90-G99', 'D50-D53', 'B00-B09', 'J85-J86', 'K40-K

In [18]:
with pd.ExcelWriter("../data/interim/diagnosticos_encontrados.xlsx") as file:
    for nombre_archivo, df_a_guardar in resultados.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_archivo)